In [16]:
import cv2
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, Concatenate

In [2]:
fecha_inicio = '2022-02-23 05:00:00'
fecha_fin = '2022-02-23 18:20:00'

image_folder = 'Images/ACMF' 

df = pd.read_pickle('df_GHI.plk')
df_filtrado = df[(df['Timestamp'] >= fecha_inicio) & (df['Timestamp'] <= fecha_fin)].copy()
df_filtrado

,Timestamp,GHI
210,2022-02-23 05:00:00,0.0080
211,2022-02-23 05:10:00,0.0264
212,2022-02-23 05:20:00,0.0449
213,2022-02-23 05:30:00,0.0636
214,2022-02-23 05:40:00,0.0821
...,...,...
286,2022-02-23 17:40:00,23.7602
287,2022-02-23 17:50:00,11.2535
288,2022-02-23 18:00:00,3.3461
289,2022-02-23 18:10:00,0.4372


In [3]:
# Función para convertir el formato de timestamp del DataFrame al formato de los nombres de las imágenes
def timestamp_to_image_name(timestamp):
    # Convertir el timestamp a 'ACMF_YYYYMMDD_HHMMSS.jpg'
    return 'ACMF_' + timestamp.strftime('%Y%m%d_%H%M%S') + '.jpg'

# Cargar imágenes y alinearlas con el DataFrame
def load_images_and_ghi(df, image_folder):
    images = []
    ghi_values = []
    
    for idx, row in df.iterrows():
        timestamp = row['Timestamp']
        ghi_value = row['GHI']
        image_name = timestamp_to_image_name(timestamp)  # Obtener el nombre de la imagen
        image_path = f'{image_folder}/{image_name}'      # Ruta completa de la imagen
        img = cv2.imread(image_path)
        
        if img is not None:
            img_resized = cv2.resize(img, (64, 64))  # Redimensionar a 64x64
            images.append(img_resized)
            ghi_values.append(ghi_value)
        else:
            print(f"Imagen no encontrada: {image_path}")
    
    return np.array(images), np.array(ghi_values)

# Función para crear secuencias de imágenes y GHI
def create_sequences(X_images, y_ghi, seq_len, num_pred):
    X_seq_images = []
    X_seq_ghi = []
    y_seq_ghi = []

    for i in range(len(X_images) - seq_len - num_pred):
        X_seq_images.append(X_images[i:i+seq_len])  # Secuencia de imágenes
        X_seq_ghi.append(y_ghi[i:i+seq_len])        # Secuencia de valores GHI
        y_seq_ghi.append(y_ghi[i+seq_len:i+seq_len+num_pred])  # Valores a predecir

    return np.array(X_seq_images), np.array(X_seq_ghi), np.array(y_seq_ghi)

In [4]:
# Cargar las imágenes redimensionadas y los valores GHI
X_images, y_ghi = load_images_and_ghi(df_filtrado, image_folder)

In [5]:
seq_len = 60  # Longitud de la secuencia de entrada (71 imágenes)
num_pred = 10  # Número de valores GHI a predecir

# Crear secuencias de entrada y salida con 71 imágenes de entrada y 10 valores a predecir
X_images_seq, X_ghi_seq, y_ghi_pred = create_sequences(X_images, y_ghi, seq_len, num_pred)


In [12]:
# Parámetros del modelo
img_shape = (64, 64, 3)  # Tamaño de las imágenes redimensionadas (64x64)
seq_len = 60  # Longitud de la secuencia de entrada (71 imágenes)
num_pred = 10  # Número de valores a predecir

# Entrada de imágenes
image_input = Input(shape=(seq_len, *img_shape))

# CNN para procesar cada imagen en la secuencia (TimeDistributed aplica la CNN a cada imagen de la secuencia)
cnn_model = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(image_input)
cnn_model = TimeDistributed(MaxPooling2D((2, 2)))(cnn_model)
cnn_model = TimeDistributed(Conv2D(64, (3, 3), activation='relu'))(cnn_model)
cnn_model = TimeDistributed(MaxPooling2D((2, 2)))(cnn_model)
cnn_model = TimeDistributed(Conv2D(128, (3, 3), activation='relu'))(cnn_model)
cnn_model = TimeDistributed(MaxPooling2D((2, 2)))(cnn_model)
cnn_model = TimeDistributed(Flatten())(cnn_model)

# Entrada de los valores numéricos (GHI)
numeric_input = Input(shape=(seq_len, 1))

# Concatenar la salida de la CNN y los valores GHI
concat = Concatenate()([cnn_model, numeric_input])

# LSTM para capturar la dependencia temporal
lstm_out = LSTM(64, return_sequences=False)(concat)

# Capa densa para generar la predicción final (10 valores hacia adelante)
output = Dense(num_pred)(lstm_out)

# Definir el modelo
model = Model([image_input, numeric_input], output)

# Compilar el modelo
model.compile(optimizer='adam', loss='mse')

# Resumen del modelo
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 60, 64, 64, 3)]      0         []                            
                                                                                                  
 time_distributed_21 (TimeD  (None, 60, 62, 62, 32)       896       ['input_7[0][0]']             
 istributed)                                                                                      
                                                                                                  
 time_distributed_22 (TimeD  (None, 60, 31, 31, 32)       0         ['time_distributed_21[0][0]'] 
 istributed)                                                                                      
                                                                                            

In [13]:
# Entrenar el modelo con secuencias de 71 imágenes y valores GHI
model.fit([X_images_seq, X_ghi_seq], y_ghi_pred, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 126531.0000 - val_loss: 21563.0469
Epoch 2/50
1/1 [==============================] - 0s 335ms/step - loss: 126150.1094 - val_loss: 21539.6543
Epoch 3/50
1/1 [==============================] - 0s 371ms/step - loss: 126097.6172 - val_loss: 21530.1504
Epoch 4/50
1/1 [==============================] - 0s 335ms/step - loss: 126068.8125 - val_loss: 21515.9980
Epoch 5/50
1/1 [==============================] - 0s 374ms/step - loss: 126029.1484 - val_loss: 21479.5312
Epoch 6/50
1/1 [==============================] - 0s 334ms/step - loss: 125973.7031 - val_loss: 21475.0215
Epoch 7/50
1/1 [==============================] - 0s 347ms/step - loss: 125960.7812 - val_loss: 21470.3379
Epoch 8/50
1/1 [==============================] - 0s 331ms/step - loss: 125947.3594 - val_loss: 21465.5410
Epoch 9/50
1/1 [==============================] - 0s 334ms/step - loss: 125933.5781 - val_loss: 21458.9219
Epoch 10/50
1/1 [=======================

In [15]:
# Graficar los valores reales y predichos
def plot_predictions(y_true, y_pred, title="Predicciones vs Valores Reales"):
    plt.figure(figsize=(10, 6))
    
    # Valores reales
    plt.plot(y_true, label="Valores Reales", color='blue', marker='o')
    
    # Valores predichos
    plt.plot(y_pred, label="Valores Predichos", color='red', linestyle='--', marker='x')
    
    plt.title(title)
    plt.xlabel('Índice de Tiempo')
    plt.ylabel('GHI')
    plt.legend()
    plt.grid(True)
    plt.show()

# Hacer predicciones con el modelo usando el conjunto de prueba
y_ghi_pred_test = model.predict([X_images_seq_test, X_ghi_seq_test])

# Suponiendo que tienes los valores reales en y_ghi_test
plot_predictions(y_ghi_test, y_ghi_pred_test)

NameError: name 'X_images_seq_test' is not defined